# Lab 11 - Association Rule Learning - Apriori Algorithm

During this lab we will explore association rule learning. It is a domain of data mining that
focuses on discovering interesting relationships between variables in transactional data. You will
familiarize yourself with basic concepts such as association rules, support, confidence, lift, and
leverage and you will implement the Apriori algorithm.

You can also deepen your understanding and knowledge by studying the relevant materials from
[Chapter 6 (pdf)](http://infolab.stanford.edu/~ullman/mmds/ch6.pdf) of "Mining of Massive Datasets"
\- http://www.mmds.org/.

## 1. The Dataset

We will use a dataset that represents transactions of customers from the Instacart online grocery
delivery platform. The "The Instacart Online Grocery Shopping Dataset 2017" was provided on the
Kaggle platform. Although it is not available at its original location, you can find the files at:
- https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset
- or 
- https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis
- or 
- https://www.kaggle.com/datasets/suhasyogeshwara/instcart-market-analysis

Download the dataset, extract the files and familiarize yourself with the data.

There are 6 files available:
- aisles.csv - aisles of the store
- departments.csv - departments of the store
- order_products__prior.csv - details of prior orders (historical data)
- order_products__train.csv - details of train orders (last order for each customer)
- orders.csv - orders of the customers
- products.csv - products of the store

In [ ]:
# write your code here


### 1.1 Products and Transactions

We are not going to use the whole information provided by the dataset. For now, we are interested in
the list of products that customers bought in each transaction. Let's focus on the
`order_products__prior.csv` (historical transactions data) and `products.csv` (for the names of the
products) files.

For example:

```python
>>> order_products_prior_df[order_products_prior_df["order_id"] == 2]

    order_id  product_id  add_to_cart_order  reordered
59         7       34050                  1          0
60         7       46802                  2          0
```

We can see that a customer bought both products (with id `34050` and `46802`) in one transaction. 

If we want to check the names of the products, we can use the `products.csv` file:

```python
>>> products_df[products_df["product_id"].isin([34050, 46802])]

       product_id      product_name  aisle_id  department_id
34049       34050      Orange Juice        31              7
46801       46802  Pineapple Chunks       116              1
```

In [36]:
# write your code here


### 1.2 Association Rules

Association rules are rules that express the relationship between items in transactions. They are
usually presentented in the form `A -> B`, where `A` and `B` are subsets of items, and `A -> B`
means that if `A` is purchased, then `B` is also purchased. We will refer to `A` as the antecedent
and `B` as the consequent of the rule.

There are several metrics that can be used to evaluate the quality of association rules. The most
common metrics are:
- support, 
- confidence, 
- lift, 
- leverage.

In [ ]:
# write your code here
